In [1]:
import matplotlib.pyplot as plt
from scipy import optimize
import numpy as np
import sympy as sm
import func

In [2]:
# Parameters
#Micro:
theta = 0.5
beta = 0.9
rho = 0.5
#Macro:
alpha = 1/3
delta = 0.05
n=0.008

# produktion:
b = 10
k0 = 10
l = [(1+n)**i for i in range(t)]
